In [1]:
import pandas as pd
import numpy as np
import datetime
import requests
import json
import os
import time
import ftplib
from pyproj import Transformer
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from dotenv import load_dotenv

In [2]:
heure_debut=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
dir_api = "./api/"
if not os.path.exists(dir_api):
    os.makedirs(dir_api)
dir_data = "./data/"
if not os.path.exists(dir_data):
    os.makedirs(dir_data)
dir_output = "./output/"
if not os.path.exists(dir_output):
    os.makedirs(dir_output)
dir_apiinsee = "./api/"+datetime.now().strftime('%Y%m%d')
if not os.path.exists(dir_apiinsee):
    os.makedirs(dir_apiinsee)

In [3]:
load_dotenv()
apikey = os.getenv("apikey")
inseekey = os.getenv("inseekey")
apikeyods = os.getenv("apikeyods")
ftp_HOSTNAME = os.getenv("ftp_HOSTNAME")
ftp_USERNAME = os.getenv("ftp_USERNAME")
ftp_PASSWORD = os.getenv("ftp_PASSWORD")

In [4]:
categoriesJuridiques = pd.read_excel(dir_data+'categorieJuridiqueUniteLegale.xlsx',dtype=str)
categoriesJuridiques=categoriesJuridiques[['categorieJuridiqueUniteLegale','categorieJuridiqueUniteLegaleLib']]
activitePrincipaleUniteLegale = pd.read_excel(dir_data+'activitePrincipaleUniteLegale.xlsx',dtype=str)
maj_structures_sirene_prec=pd.read_parquet(dir_data+"maj_structures_sirene_prec.parquet", engine='pyarrow')

In [5]:
les_communes=pd.read_parquet(dir_data+"les_communes.parquet", engine='pyarrow')

In [6]:
headers = {
    'Accept': 'application/json',
    'X-API-KEY': apikey,
}
try:
    response = requests.get("https://api.paysage.dataesr.ovh/identifiers?filters[type]=siret&limit=10000", headers=headers)
    response.raise_for_status()
    result = response.json()
    sirets = pd.DataFrame(result["data"])
    time.sleep(.6)
except requests.exceptions.HTTPError as errh:
    print("siret > KO : ", response.status_code)
sirets=sirets.rename(columns={"value": "siret"})
sirets["dernier_traitement"]=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
sirets["url_paysage"]="https://paysage.enseignementsup-recherche.gouv.fr/structures/"+sirets["resourceId"]+"/presentation"
sirets["url_annuaire"]="https://annuaire-entreprises.data.gouv.fr/etablissement/"+sirets["siret"]
sirets["url_ods"]="https://data.enseignementsup-recherche.gouv.fr/explore/dataset/fr-esr-maj_structures_sirene/table/?sort=datedebut&refine.numero_siren="+sirets["siret"].str[:9]
les_sirets=sirets[(~sirets["siret"].isnull())]["siret"].drop_duplicates()
siret_paysage = sirets.copy()
siret_paysage["Siret_dans_paysage"] = "Oui"
siret_paysage=siret_paysage[(~siret_paysage["siret"].isnull())][["siret","Siret_dans_paysage"]].drop_duplicates()
siret_paysage=siret_paysage.rename(columns={"siret": "dern_numero_siret"})
les_sirens=les_sirets.str[:9].drop_duplicates()
les_sirens
siret_paysage2 = sirets.copy()
siret_paysage2["siren"]=siret_paysage2["siret"].str[:9]
siret_paysage2=siret_paysage2[["resourceId","siren"]].drop_duplicates()
siret_paysage2

,resourceId,siren
0,9unA5,799066618
1,zh675,390220119
2,br3Cc,538758236
3,emqo9,497511923
4,xYJ6W,879223923
...,...,...
6301,xJdyB,130015332
6302,1tI7C,191333400
6303,8tVLr,191010602
6304,7CYWd,190609313


In [7]:
siren_paysage = (siret_paysage2.pivot_table(index=['siren'], values='resourceId', aggfunc=';'.join)
            .rename_axis(columns=None)
         ).reset_index()
siren_paysage['nb'] = siren_paysage["resourceId"].str.count(';')+1
siren_paysage_unique=siren_paysage[(siren_paysage['nb']==1)]

In [8]:
headers = {
    'Accept': 'application/json',
    'X-API-KEY': apikey,
}
try:
    response = requests.get("https://api.paysage.dataesr.ovh/identifiers?filters[type]=rna&limit=10000", headers=headers)
    response.raise_for_status()
    result = response.json()
    rnas = pd.DataFrame(result["data"])
    time.sleep(.6)
except requests.exceptions.HTTPError as errh:
    print("siret > KO : ", response.status_code)
rnas=rnas.rename(columns={"value": "identifiantAssociationUniteLegale"})
rnas["Rna_dans_paysage"] = "Oui"
Rna_paysage=rnas[(~rnas["identifiantAssociationUniteLegale"].isnull())][["identifiantAssociationUniteLegale","Rna_dans_paysage"]].drop_duplicates()
Rna_paysage

,identifiantAssociationUniteLegale,Rna_dans_paysage
0,W751276301,Oui
1,W133036847,Oui
2,W353021707,Oui
3,W313004321,Oui
4,W751265632,Oui
...,...,...
196,W601001426,Oui
197,W595008501,Oui
198,W595026827,Oui
199,W543003821,Oui


In [9]:
headers = {
    'X-INSEE-Api-Key-Integration': inseekey,
}
def extract_maj_sirene(dateheure,curseur="*"):
    global i
    i=i+1
    les_sirets_modifs = pd.DataFrame(columns = ["siret","dateDernierTraitementEtablissement"])
    cursor = curseur.replace("*","-")
    dateheure2 = datetime.now()
    dateheure2 = dateheure2.isoformat()
    dateheure2 = dateheure2[:13]
    params = {
        'q': 'dateDernierTraitementEtablissement:['+dateheure+' TO '+dateheure2+']',
        'champs': 'siret,dateDernierTraitementEtablissement',
        'nombre': '1000',
        'curseur': curseur
    }
    if os.path.isfile(dir_apiinsee+"/modifs_-_"+cursor+".json"):
        f = open(dir_apiinsee+"/modifs_-_"+cursor+".json")
        result = json.load(f)
        f.close()
    else:
        response = requests.get('https://api.insee.fr/api-sirene/3.11/siret', params=params, headers=headers)
        writeFile =open(dir_apiinsee+"/modifs_-_"+cursor+".json", 'w')
        json.dump(response.json(),writeFile)
        writeFile.close()
        result = response.json()
        time.sleep(2.1)
    if result["header"]["statut"]==200:
        les_sirets_modif = pd.DataFrame(result["etablissements"])
        les_sirets_modifs = pd.concat([les_sirets_modifs, les_sirets_modif], axis=0, join='outer')
        curseurSuivant=result["header"]["curseurSuivant"]
        if curseurSuivant != curseur :
            if i==1 :
                print("Total : "+str(result["header"]["total"]))
            if i%25==0 :
                print(str(i)+"000 - "+datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            les_sirets_modif = extract_maj_sirene(dateheure,curseurSuivant)
            les_sirets_modif = pd.DataFrame(les_sirets_modif)
            les_sirets_modifs = pd.concat([les_sirets_modifs, les_sirets_modif], axis=0, join='outer')
        return les_sirets_modifs    

In [10]:
#debut = datetime.now() - relativedelta(days=6)
#fin = datetime.now() + relativedelta(hours=1)
#delta = timedelta(hours=1)
#dates = []
#while debut <= fin:
#    dates.append(debut.isoformat())
#    debut += delta
#i=0
#for date in dates:
les_sirets_modifs = pd.DataFrame(columns = ["siret","dateDernierTraitementEtablissement"])

In [11]:
i=0
les_sirets_modifs = pd.DataFrame(columns = ["siret","dateDernierTraitementEtablissement"])
dateheure = datetime.now() - relativedelta(days=10)
dateheure = dateheure.isoformat()
dateheure = dateheure[:13]
les_sirets_modif = extract_maj_sirene(dateheure)
les_sirets_modif = pd.DataFrame(les_sirets_modif)
les_sirets_modifs = pd.concat([les_sirets_modifs, les_sirets_modif], axis=0, join='outer')
les_sirets_modifs=les_sirets_modifs.drop_duplicates()
print("Terminé")

Total : 161310
25000 - 2024-12-05 14:09:17
50000 - 2024-12-05 14:09:17
75000 - 2024-12-05 14:09:17
100000 - 2024-12-05 14:09:17
125000 - 2024-12-05 14:09:17
150000 - 2024-12-05 14:09:17
Terminé


In [12]:
print(les_sirets_modifs)

              siret dateDernierTraitementEtablissement
0    00562003400032            2024-12-04T05:34:13.249
1    00648024800043            2024-12-03T10:41:38.928
2    00648024800050            2024-12-03T10:41:38.933
3    00652026600039            2024-12-04T05:34:06.253
4    00665008900037            2024-11-27T08:25:45.849
..              ...                                ...
305  99882350433176            2024-12-03T15:59:08.460
306  99889300200142            2024-11-26T17:05:49.739
307  99889300200217            2024-11-29T17:02:46.993
308  99999030200047            2024-11-26T05:31:34.206
309  99999036900095            2024-12-02T10:31:49.640

[161310 rows x 2 columns]


In [13]:
les_siretss=pd.DataFrame(les_sirets)
les_siretss=les_siretss.rename(columns={0: "siret"})
les_sirets_a_gerer=pd.merge(les_siretss,les_sirets_modifs,how='inner',on=['siret'])
les_sirets_a_gerer

,siret,dateDernierTraitementEtablissement
0,39022011900049,2024-12-02T17:16:27.771
1,53875823600052,2024-12-02T09:54:12.037
2,49751192300052,2024-12-02T10:44:40.671
3,87922392300035,2024-11-29T12:01:02.192
4,89241456600042,2024-11-28T09:26:25.153
...,...,...
69,77566410500019,2024-12-03T09:53:22.098
70,18004619500012,2024-11-28T05:25:53.003
71,19624401600016,2024-11-30T05:23:15.127
72,13001727000146,2024-11-26T05:25:13.744


In [14]:
nbfic = 0
for ind in les_sirets_a_gerer.index:
    numero_siret = les_sirets_a_gerer['siret'][ind]
    numero_siren= numero_siret[:9]
    fname = dir_api+"/"+numero_siret+".json"
    if os.path.isfile(fname):
        os.remove(fname)
        nbfic+=1
    fname = dir_api+"/"+numero_siren+".json"
    if os.path.isfile(fname):
        os.remove(fname)
        nbfic+=1
if nbfic>1:
    print(str(nbfic)+" fichier(s) json supprimé(s)")
else:
    print(str(nbfic)+" fichier json supprimé")

89 fichier(s) json supprimé(s)


In [15]:
headers = {
    'X-INSEE-Api-Key-Integration': inseekey,
}
def extract_siren(numero_siren):
    structure = {}
    structure["numero_siren"] = numero_siren
    sep=";"
    params = {
        'q': 'siren:'+numero_siren,
        'masquerValeursNulles' : 'true'
    }
    if os.path.isfile(dir_api+"/"+numero_siren+".json"):
        f = open(dir_api+"/"+numero_siren+".json")
        result = json.load(f)
        f.close()
    else:
        response = requests.get('https://api.insee.fr/api-sirene/3.11/siren', params=params, headers=headers)
        writeFile =open(dir_api+"/"+numero_siren+".json", 'w')
        json.dump(response.json(),writeFile)
        writeFile.close()
        result = response.json()
        time.sleep(2.02)
    structure["statut"] = result["header"]["statut"]
    if result["header"]["statut"]==200:
        if "unitesLegales" in result:
            if "dateCreationUniteLegale" in result["unitesLegales"][0]:
                structure["dateCreationUniteLegale"]=result["unitesLegales"][0]["dateCreationUniteLegale"]
            if "identifiantAssociationUniteLegale" in result["unitesLegales"][0]:
                structure["identifiantAssociationUniteLegale"]=result["unitesLegales"][0]["identifiantAssociationUniteLegale"]
            if "sigleUniteLegale" in result["unitesLegales"][0]:
                structure["sigleUniteLegale"]=result["unitesLegales"][0]["sigleUniteLegale"]
            if "dateDernierTraitementUniteLegale" in result["unitesLegales"][0]:
                structure["dateDernierTraitementUniteLegale"]=result["unitesLegales"][0]["dateDernierTraitementUniteLegale"]
            if "categorieEntreprise" in result["unitesLegales"][0]:
                structure["categorieEntreprise"]=result["unitesLegales"][0]["categorieEntreprise"]
            if result["unitesLegales"][0]["nombrePeriodesUniteLegale"]>0:
                if "dateDebut" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["dateDebutDerniere"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["dateDebut"]
                if "dateFin" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["dateFinDerniere"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["dateFin"]
                if "denominationUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["denominationUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["denominationUniteLegale"]
                if "nomUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["nomUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["nomUniteLegale"]
                if "nomUsageUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["nomUsageUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["nomUsageUniteLegale"]
                if "denominationUsuelle1UniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["denominationUsuelle1UniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["denominationUsuelle1UniteLegale"]
                if "denominationUsuelle2UniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["denominationUsuelle2UniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["denominationUsuelle2UniteLegale"]
                if "denominationUsuelle3UniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["denominationUsuelle3UniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["denominationUsuelle3UniteLegale"]
                if "etatAdministratifUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["etatAdministratifUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["etatAdministratifUniteLegale"]
                if "nicSiegeUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["nicSiegeUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["nicSiegeUniteLegale"]
                    structure["numero_siret"]=numero_siren+result["unitesLegales"][0]["periodesUniteLegale"][0]["nicSiegeUniteLegale"]
                if "categorieJuridiqueUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["categorieJuridiqueUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["categorieJuridiqueUniteLegale"]
                if "activitePrincipaleUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["activitePrincipaleUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["activitePrincipaleUniteLegale"]
                if "nomenclatureActivitePrincipaleUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["nomenclatureActivitePrincipaleUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["nomenclatureActivitePrincipaleUniteLegale"]
                if "changementEtatAdministratifUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["changementEtatAdministratifUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["changementEtatAdministratifUniteLegale"]
                if "changementNomUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["changementNomUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["changementNomUniteLegale"]
                if "changementNomUsageUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["changementNomUsageUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["changementNomUsageUniteLegale"]
                if "changementDenominationUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["changementDenominationUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["changementDenominationUniteLegale"]
                if "changementDenominationUsuelleUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["changementDenominationUsuelleUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["changementDenominationUsuelleUniteLegale"]
                if "changementCategorieJuridiqueUniteLegale" in result["unitesLegales"][0]["periodesUniteLegale"][0]:
                    structure["changementCategorieJuridiqueUniteLegale"]=result["unitesLegales"][0]["periodesUniteLegale"][0]["changementCategorieJuridiqueUniteLegale"]
                identifiers = pd.DataFrame(result["unitesLegales"][0]["periodesUniteLegale"])
                if identifiers["categorieJuridiqueUniteLegale"].count() > 0:
                    categoriesJuridiques=identifiers["categorieJuridiqueUniteLegale"].dropna().unique()
                    structure["categoriesJuridiques"] = sep.join(map(str, categoriesJuridiques))
                    structure["categoriesJuridiquesCompte"] = len(categoriesJuridiques)
    return structure

In [16]:
structures=pd.DataFrame(columns = ["numero_siren","dateCreationUniteLegale"])
i=0
for ind in les_sirens.index:
    numero_siren = les_sirens[ind]
    i=i+1
    if i%250==0:
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S')+" "+str(i)+"/{:0.1f} %".format(i*100/len(les_sirens.index)))
    structure = extract_siren(numero_siren)
    structure = pd.DataFrame(structure, index=[0])
    if (structures.empty) & (not structure.empty):
        structures = structure
    elif not structure.empty:
        structures = pd.concat([structures, structure], axis=0, join='outer')
print("Terminé")

2024-12-05 14:09:50 250/4.8 %
2024-12-05 14:10:05 500/9.6 %
2024-12-05 14:10:16 750/14.4 %
2024-12-05 14:10:20 1000/19.2 %
2024-12-05 14:10:24 1250/24.0 %
2024-12-05 14:10:26 1500/28.8 %
2024-12-05 14:10:27 1750/33.6 %
2024-12-05 14:10:31 2000/38.4 %
2024-12-05 14:10:44 2250/43.3 %
2024-12-05 14:10:52 2500/48.1 %
2024-12-05 14:11:04 2750/52.9 %
2024-12-05 14:11:07 3000/57.7 %
2024-12-05 14:11:24 3250/62.5 %
2024-12-05 14:11:35 3500/67.3 %
2024-12-05 14:11:42 3750/72.1 %
2024-12-05 14:11:46 4000/76.9 %
2024-12-05 14:11:55 4250/81.7 %
2024-12-05 14:12:05 4500/86.5 %
2024-12-05 14:12:18 4750/91.3 %
2024-12-05 14:12:25 5000/96.1 %
Terminé


In [17]:
structures

,numero_siren,statut,dateCreationUniteLegale,dateDernierTraitementUniteLegale,categorieEntreprise,dateDebutDerniere,denominationUniteLegale,etatAdministratifUniteLegale,nicSiegeUniteLegale,numero_siret,...,changementDenominationUsuelleUniteLegale,changementCategorieJuridiqueUniteLegale,categoriesJuridiques,categoriesJuridiquesCompte,sigleUniteLegale,identifiantAssociationUniteLegale,denominationUsuelle1UniteLegale,denominationUsuelle2UniteLegale,nomUniteLegale,nomUsageUniteLegale
0,799066618,200,2013-12-06,2024-11-22T14:05:38.802,PME,2024-11-01,E-PHY-SCIENCE,A,00031,79906661800031,...,False,False,5499;5710,2.0,NaN,NaN,NaN,NaN,NaN,NaN
0,390220119,200,1993-01-29,2024-12-02T17:16:27.790,PME,2024-11-20,SCIENCES & COMPUTERS CONSULTANTS,A,00049,39022011900049,...,False,False,5499;5710,2.0,S&CC,NaN,NaN,NaN,NaN,NaN
0,538758236,200,2011-12-12,2024-12-02T09:54:12.070,PME,2024-06-18,DRACULA TECHNOLOGIES,A,00052,53875823600052,...,False,False,5710;5499,2.0,NaN,NaN,NaN,NaN,NaN,NaN
0,497511923,200,2007-04-03,2024-12-02T10:44:40.698,PME,2024-04-01,WEATHERNEWS FRANCE,A,00052,49751192300052,...,False,False,5710;5599,2.0,NaN,NaN,NaN,NaN,NaN,NaN
0,879223923,200,2019-09-15,2024-11-29T12:01:02.196,PME,2024-10-24,SPORTSDYNAMICS,A,00035,87922392300035,...,False,False,5710,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,191700327,200,1993-01-28,2024-05-31T19:19:14.075,ETI,2018-10-15,LA ROCHELLE UNIVERSITE,A,00015,19170032700015,...,False,False,7383,1.0,NaN,NaN,NaN,NaN,NaN,NaN
0,130021330,200,2014-12-29,2024-03-22T14:26:06.001,PME,2014-12-29,COMMUNAUTE UNIVERSITES ET ETABLISSEMENTS NORMA...,A,00019,13002133000019,...,False,False,7383,1.0,COMUE,NaN,NaN,NaN,NaN,NaN
0,191333400,200,2003-10-01,2024-03-22T14:26:06.001,PME,2008-01-01,ECOLE CENTRALE DE MARSEILLE,A,00015,19133340000015,...,False,False,7383,1.0,CENTRALE MARSEILLE,NaN,NaN,NaN,NaN,NaN
0,190609313,200,1970-06-10,2024-03-22T14:26:06.001,NaN,2020-01-01,UNIVERSITE DE NICE,C,00019,19060931300019,...,False,False,7383,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
epsg = {'D001':'EPSG:2154','D002':'EPSG:2154','D003':'EPSG:2154','D004':'EPSG:2154','D005':'EPSG:2154','D006':'EPSG:2154','D007':'EPSG:2154','D008':'EPSG:2154','D009':'EPSG:2154','D010':'EPSG:2154','D011':'EPSG:2154','D012':'EPSG:2154','D013':'EPSG:2154','D014':'EPSG:2154','D015':'EPSG:2154','D016':'EPSG:2154','D017':'EPSG:2154','D018':'EPSG:2154','D019':'EPSG:2154','D021':'EPSG:2154','D022':'EPSG:2154','D023':'EPSG:2154','D024':'EPSG:2154','D025':'EPSG:2154','D026':'EPSG:2154','D027':'EPSG:2154','D028':'EPSG:2154','D029':'EPSG:2154','D02A':'EPSG:2154','D02B':'EPSG:2154','D030':'EPSG:2154','D031':'EPSG:2154','D032':'EPSG:2154','D033':'EPSG:2154','D034':'EPSG:2154','D035':'EPSG:2154','D036':'EPSG:2154','D037':'EPSG:2154','D038':'EPSG:2154','D039':'EPSG:2154','D040':'EPSG:2154','D041':'EPSG:2154','D042':'EPSG:2154','D043':'EPSG:2154','D044':'EPSG:2154','D045':'EPSG:2154','D046':'EPSG:2154','D047':'EPSG:2154','D048':'EPSG:2154','D049':'EPSG:2154','D050':'EPSG:2154','D051':'EPSG:2154','D052':'EPSG:2154','D053':'EPSG:2154','D054':'EPSG:2154','D055':'EPSG:2154','D056':'EPSG:2154','D057':'EPSG:2154','D058':'EPSG:2154','D059':'EPSG:2154','D060':'EPSG:2154','D061':'EPSG:2154','D062':'EPSG:2154','D063':'EPSG:2154','D064':'EPSG:2154','D065':'EPSG:2154','D066':'EPSG:2154','D067':'EPSG:2154','D068':'EPSG:2154','D069':'EPSG:2154','D070':'EPSG:2154','D071':'EPSG:2154','D072':'EPSG:2154','D073':'EPSG:2154','D074':'EPSG:2154','D075':'EPSG:2154','D076':'EPSG:2154','D077':'EPSG:2154','D078':'EPSG:2154','D079':'EPSG:2154','D080':'EPSG:2154','D081':'EPSG:2154','D082':'EPSG:2154','D083':'EPSG:2154','D084':'EPSG:2154','D085':'EPSG:2154','D086':'EPSG:2154','D087':'EPSG:2154','D088':'EPSG:2154','D089':'EPSG:2154','D090':'EPSG:2154','D091':'EPSG:2154','D092':'EPSG:2154','D093':'EPSG:2154','D094':'EPSG:2154','D095':'EPSG:2154','D971':'EPSG:5490','D972':'EPSG:5490','D973':'EPSG:2972','D974':'EPSG:2975','D975':'EPSG:4467','D976':'EPSG:4471','D978':'EPSG:5490','D986':'EPSG:8903','D987':'EPSG:6687','D988':'EPSG:3163'}
les_communes2=les_communes[(les_communes["reg_id"]!="R99")&(les_communes["reg_id"]!="R00")][["com_code","dep_id"]].rename(columns={"com_code": "codeCommuneEtablissement","dep_id": "epsg"}).drop_duplicates(subset=['codeCommuneEtablissement'])
les_communes2=les_communes2.replace({"epsg": epsg})

In [19]:
les_sirets= pd.concat([les_sirets, structures["numero_siret"].drop_duplicates()], axis=0, join='outer').drop_duplicates().reset_index()
les_sirets=les_sirets[(~les_sirets[0].isnull())][[0]].drop_duplicates()
les_sirets

,0
0,79906661800031
1,39022011900049
2,53875823600052
3,49751192300052
4,87922392300035
...,...
6428,78461701100027
6429,78475641300117
6430,75080177100023
6431,18640005700011


In [20]:
headers = {
    'X-INSEE-Api-Key-Integration': inseekey,
}
def extract_data_siret(numero_siret):
    structure = {}
    numero_siren = numero_siret[:9]
    structure["siren"] = numero_siren
    structure["siret"] = numero_siret
    sep=";"
    params = {
        'q': 'siret:'+numero_siret,
        'masquerValeursNulles' : 'true'
    }
    if os.path.isfile(dir_api+"/"+numero_siret+".json"):
        f = open(dir_api+"/"+numero_siret+".json")
        result = json.load(f)
        f.close()
    else:
        response = requests.get('https://api.insee.fr/api-sirene/3.11/siret', params=params, headers=headers)
        #response.raise_for_status()
        writeFile =open(dir_api+"/"+numero_siret+".json", 'w')
        json.dump(response.json(),writeFile)
        writeFile.close()
        result = response.json()
        time.sleep(2.02)
    structure["statut"] = result["header"]["statut"]
    structure["message"] = result["header"]["message"]
    if result["header"]["statut"]==200:
        if "etablissements" in result:
            if "etablissementSiege" in result["etablissements"][0]:
                structure["etablissementSiege"]=result["etablissements"][0]["etablissementSiege"]            
            if "dateCreationEtablissement" in result["etablissements"][0]:
                structure["dateCreationEtablissement"]=result["etablissements"][0]["dateCreationEtablissement"]            
            if "dateDernierTraitementEtablissement" in result["etablissements"][0]:
                structure["dateDernierTraitementEtablissement"]=result["etablissements"][0]["dateDernierTraitementEtablissement"]            
            if "uniteLegale" in result["etablissements"][0]:
                if "etatAdministratifUniteLegale" in result["etablissements"][0]["uniteLegale"]:
                    structure["etatAdministratifUniteLegale"]=result["etablissements"][0]["uniteLegale"]["etatAdministratifUniteLegale"]            
                if "dateCreationUniteLegale" in result["etablissements"][0]["uniteLegale"]:
                    structure["dateCreationUniteLegale"]=result["etablissements"][0]["uniteLegale"]["dateCreationUniteLegale"]            
                if "nicSiegeUniteLegale" in result["etablissements"][0]["uniteLegale"]:
                    structure["siretSiege"]=numero_siren+result["etablissements"][0]["uniteLegale"]["nicSiegeUniteLegale"]
                if "sigleUniteLegale" in result["etablissements"][0]["uniteLegale"]:
                    structure["sigleUniteLegale"]=result["etablissements"][0]["uniteLegale"]["sigleUniteLegale"]            
                if "denominationUniteLegale" in result["etablissements"][0]["uniteLegale"]:
                    structure["denominationUniteLegale"]=result["etablissements"][0]["uniteLegale"]["denominationUniteLegale"]            
                if "categorieJuridiqueUniteLegale" in result["etablissements"][0]["uniteLegale"]:
                    structure["categorieJuridiqueUniteLegale"]=result["etablissements"][0]["uniteLegale"]["categorieJuridiqueUniteLegale"]            
            if "adresseEtablissement" in result["etablissements"][0]:
                if "numeroVoieEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["numeroVoieEtablissement"]=str(result["etablissements"][0]["adresseEtablissement"]["numeroVoieEtablissement"])
                if "dernierNumeroVoieEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["dernierNumeroVoieEtablissement"]=str(result["etablissements"][0]["adresseEtablissement"]["dernierNumeroVoieEtablissement"])
                if "typeVoieEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["typeVoieEtablissement"]=result["etablissements"][0]["adresseEtablissement"]["typeVoieEtablissement"]            
                if "libelleVoieEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["libelleVoieEtablissement"]=result["etablissements"][0]["adresseEtablissement"]["libelleVoieEtablissement"]            
                if "complementAdresseEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["complementAdresseEtablissement"]=result["etablissements"][0]["adresseEtablissement"]["complementAdresseEtablissement"]            
                if "codePostalEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["codePostalEtablissement"]=str(result["etablissements"][0]["adresseEtablissement"]["codePostalEtablissement"])
                if "libelleCommuneEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["libelleCommuneEtablissement"]=result["etablissements"][0]["adresseEtablissement"]["libelleCommuneEtablissement"]            
                if "codePaysEtrangerEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["codePaysEtrangerEtablissement"]=str(result["etablissements"][0]["adresseEtablissement"]["codePaysEtrangerEtablissement"])
                if "libellePaysEtrangerEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["libellePaysEtrangerEtablissement"]=result["etablissements"][0]["adresseEtablissement"]["libellePaysEtrangerEtablissement"]            
                if "libelleCommuneEtrangerEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["libelleCommuneEtrangerEtablissement"]=result["etablissements"][0]["adresseEtablissement"]["libelleCommuneEtrangerEtablissement"]            
                if "coordonneeLambertAbscisseEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["coordonneeEtablissement"]=result["etablissements"][0]["adresseEtablissement"]["coordonneeLambertAbscisseEtablissement"]+","+result["etablissements"][0]["adresseEtablissement"]["coordonneeLambertOrdonneeEtablissement"]
                if "identifiantAdresseEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["identifiantAdresseEtablissement"]=result["etablissements"][0]["adresseEtablissement"]["identifiantAdresseEtablissement"]            
                if "codeCommuneEtablissement" in result["etablissements"][0]["adresseEtablissement"]:
                    structure["codeCommuneEtablissement"]=result["etablissements"][0]["adresseEtablissement"]["codeCommuneEtablissement"]            
            if "periodesEtablissement" in result["etablissements"][0]:
                if "dateDebut" in result["etablissements"][0]["periodesEtablissement"]:
                    structure["dateDebut"]=result["etablissements"][0]["periodesEtablissement"][0]["dateDebut"]
                if "etatAdministratifEtablissement" in result["etablissements"][0]["periodesEtablissement"]:
                    structure["etatAdministratifEtablissement"]=result["etablissements"][0]["periodesEtablissement"][0]["etatAdministratifEtablissement"]            
                if "changementDenominationUsuelleEtablissement" in result["etablissements"][0]["periodesEtablissement"]:
                    structure["changementDenominationUsuelleEtablissement"]=result["etablissements"][0]["changementDenominationUsuelleEtablissement"][0]["etatAdministratifUniteLegale"]            
                if "changementEnseigneEtablissement" in result["etablissements"][0]["periodesEtablissement"]:
                    structure["changementEnseigneEtablissement"]=result["etablissements"][0]["periodesEtablissement"][0]["changementEnseigneEtablissement"]            
                if "changementEtatAdministratifEtablissement" in result["etablissements"][0]["periodesEtablissement"]:
                    structure["changementEtatAdministratifEtablissement"]=result["etablissements"][0]["periodesEtablissement"][0]["changementEtatAdministratifEtablissement"]            
    return structure

In [21]:
numero_siret

'19941111700013'

In [ ]:
structures=pd.DataFrame(columns = ["siren","siretSiege","siret"])
i=0
for ind in les_sirets.index:
    numero_siret = les_sirets[0][ind]
    if i%250==0:
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S')+" "+str(i)+"/{:0.1f} %".format(i*100/len(les_sirets.index)))
    i=i+1
    structure = extract_data_siret(numero_siret)
    structure = pd.DataFrame(structure, index=[0])
    if (structures.empty) & (not structure.empty):
        structures = structure
    elif not structure.empty:
        structures = pd.concat([structures, structure], axis=0, join='outer')
print("Terminé")

2024-12-05 14:12:28 0/0.0 %
2024-12-05 14:12:57 250/3.9 %
2024-12-05 14:13:11 500/7.8 %
2024-12-05 14:13:21 750/11.7 %


In [ ]:
dateDernierTraitementEtablissement=structures[['siren','dateDernierTraitementEtablissement']].drop_duplicates("siren")
dateDernierTraitementEtablissement=dateDernierTraitementEtablissement.rename(columns={"siren": "numero_siren"})
structures

In [ ]:
structures=pd.merge(structures,les_communes2,how='left',on='codeCommuneEtablissement')

In [ ]:
coordonneeEtablissement=structures[(~structures["coordonneeEtablissement"].isnull())&(structures["coordonneeEtablissement"]!="[ND],[ND]")][["siret","epsg","coordonneeEtablissement"]]
coordonneeEtablissement[['X', 'Y']] = coordonneeEtablissement['coordonneeEtablissement'].str.split(',', n=1, expand=True)
coordonneeEtablissement

In [ ]:
selection=(coordonneeEtablissement["X"].astype(float) < 1000) & (coordonneeEtablissement["Y"].astype(float) < 1000)
coordonneeEtablissement.loc[selection, 'epsg']="EPSG:4326"
coordonneeEtablissement

In [ ]:
coordonneeEtablissement["epsg"].drop_duplicates()

In [ ]:
def convertGPS(dftemp,epsg):
    df=dftemp.copy()
    xx, yy = Transformer.from_crs(epsg,"EPSG:4326",always_xy=True).transform(df["X"].values, df["Y"].values)
    df["XX"]=xx
    df["YY"]=yy
    if epsg=="EPSG:4326":
        df["coordonneeEtablissementGPS"]=df["XX"].astype(str)+","+df["YY"].astype(str)
    else :
        df["coordonneeEtablissementGPS"]=df["YY"].astype(str)+","+df["XX"].astype(str)
    return df.drop_duplicates("siret")
coordonneeEtablissementGPS=pd.DataFrame(columns = ["siret","epsg","coordonneeEtablissement","X","Y","XX","YY","coordonneeEtablissementGPS"])
for epsg in coordonneeEtablissement["epsg"].drop_duplicates():
    A=convertGPS(coordonneeEtablissement[(coordonneeEtablissement["epsg"]==epsg)],epsg)
    coordonneeEtablissementGPS = pd.concat([coordonneeEtablissementGPS, A], axis=0, join='outer')
coordonneeEtablissementGPS

In [ ]:
plt.scatter(x=coordonneeEtablissementGPS['XX'], y=coordonneeEtablissementGPS['YY'])
plt.show()

In [ ]:
coordonneeEtablissementGPS=coordonneeEtablissementGPS[["siret","coordonneeEtablissementGPS"]]
structures=pd.merge(structures,coordonneeEtablissementGPS,how='left',on='siret')

In [ ]:
structures["Adresse"]=structures[['complementAdresseEtablissement','numeroVoieEtablissement','typeVoieEtablissement','libelleVoieEtablissement','codePostalEtablissement','libelleCommuneEtablissement']].agg(lambda x:' '.join(x.dropna().astype(str)), axis=1)
structures["Adresse"]

In [ ]:
structures

In [ ]:
writer = pd.ExcelWriter(dir_output+"structures_"+datetime.now().strftime('%Y-%m-%d')+".xlsx", engine='xlsxwriter')
structures.to_excel(writer,sheet_name="data", index=False)
writer.close()

In [ ]:
les__structures=pd.merge(sirets,structures,how='left',on=['siret'])
les__adresse=les__structures[["siret","Adresse","coordonneeEtablissementGPS"]].drop_duplicates('siret')
les__adresse=les__adresse.rename(columns={"siret": "dern_numero_siret"})
les__structures

In [ ]:
filename_in = dir_output+"les_structures_"+datetime.now().strftime('%Y-%m-%d')+".xlsx"
writer = pd.ExcelWriter(filename_in, engine='xlsxwriter')
les__structures.to_excel(writer,sheet_name="data", index=False)
writer.close()
ftp_server = ftplib.FTP(ftp_HOSTNAME, ftp_USERNAME, ftp_PASSWORD)
filename_out = "les_structures.xlsx"
with open(filename_in, "rb") as file:
    ftp_server.storbinary(f"STOR {filename_out}", file)
print(ftp_server.dir())
ftp_server.quit()

In [ ]:
siret_a_fermer=les__structures[((les__structures["active"]!=False)&(les__structures["etatAdministratifUniteLegale"]=="C"))]
siret_a_fermer=siret_a_fermer.sort_values('dateDernierTraitementEtablissement',ascending=False)
filename_in = dir_output+"siret_a_fermer_"+datetime.now().strftime('%Y-%m-%d')+".xlsx"
writer = pd.ExcelWriter(filename_in, engine='xlsxwriter')
siret_a_fermer.to_excel(writer,sheet_name="data", index=False)
writer.close()
ftp_server = ftplib.FTP(ftp_HOSTNAME, ftp_USERNAME, ftp_PASSWORD)
filename_out = "siret_a_fermer.xlsx"
with open(filename_in, "rb") as file:
    ftp_server.storbinary(f"STOR {filename_out}", file)
print(ftp_server.dir())
ftp_server.quit()
siret_a_fermer

In [ ]:
response = requests.get(
    'https://mesr.opendatasoft.com/api/automation/v1.0/datasets/?limit=999', 
    headers={"Authorization": "apikey "+apikeyods})
result=response.json()
df = pd.DataFrame(result["results"])
od = df[df['dataset_id'].str.contains('siret')]
od

In [ ]:
dataset_uid="da_pcgcmx"

In [ ]:
try:
    responseF = requests.post('https://mesr.opendatasoft.com/api/automation/v1.0/datasets/'+dataset_uid+'/publish/', headers={"Authorization": "apikey "+apikeyods})
    responseF.raise_for_status()
except requests.exceptions.HTTPError as errh:
    print(dataset_uid+" > KO : publish ", responseF.status_code)
print("fin")

In [ ]:
headers = {
    'Accept': 'application/json',
    'X-API-KEY': apikey,
}
try:
    response = requests.get("https://api.paysage.dataesr.ovh/legal-categories?limit=2000", headers=headers)
    response.raise_for_status()
    result = response.json()
    legcat = pd.DataFrame(result["data"])
    time.sleep(.6)
except requests.exceptions.HTTPError as errh:
    print("legcat > KO : ", response.status_code)
legcat

In [ ]:
legcat=legcat[["id","longNameFr","inseeCode","acronymFr","sector","legalPersonality"]]

In [ ]:
def extract_legcat(id,inseeCode):
    try:
        response = requests.get("https://api.paysage.dataesr.ovh/relations?filters[relationTag]=structure-categorie-juridique&filters[relatedObjectId]="+id+"&limit=2000&sort=-startDate", headers=headers)
        response.raise_for_status()
        result = response.json()
        time.sleep(.75)
    except requests.exceptions.HTTPError as errh:
        print("legcat > KO : ", response.status_code)
    if result["totalCount"]>0 :
        legcat = pd.json_normalize(result["data"], 
                     meta=['relatedObjectId','active','startDate','endDate','resourceId','updatedAt', ['resource', 'creationDate'], ['resource', 'closureDate'], ['resource', 'currentLocalisation', 'cityId'], ['resource', 'currentLocalisation', 'iso3']])
        legcat=legcat.drop(columns='id')
        legcat=legcat.rename(columns={"relatedObjectId": "id"})
        legcat["inseeCode"]=inseeCode
        return(legcat)

In [ ]:
structures_legcat=pd.DataFrame(columns = ["id"])
i=0
for ind in legcat.index:
    id = legcat["id"][ind]
    inseeCode = legcat["inseeCode"][ind]
    i=i+1
    if i%50==0:
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S')+" "+str(i)+"/{:0.1f} %".format(i*100/len(legcat.index)))
    structure_legcat = extract_legcat(id,inseeCode)
    structures_legcat = pd.concat([structures_legcat, structure_legcat], axis=0, join='outer')
print("Terminé")

In [ ]:
structures_legcat

In [ ]:
structures_legcat=structures_legcat[["id","resourceId","startDate","endDate","endDatePrevisional","active","resource.displayName","resource.creationDate","resource.closureDate","resource.structureStatus","resource.currentLocalisation.cityId","resource.currentLocalisation.iso3"]]
catjur = pd.merge(legcat,structures_legcat,how='inner',on=['id'])
filename_in = dir_output+"structures_legcat_"+datetime.now().strftime('%Y-%m-%d')+".xlsx"
writer = pd.ExcelWriter(filename_in, engine='xlsxwriter')
catjur.to_excel(writer,sheet_name="data", index=False)
writer.close()

In [ ]:
sirets2=sirets[['resourceId','siret','startDate','endDate','active']]
sirets2=sirets2.rename(columns={'startDate':'startDate_siret','endDate':'endDate_siret','active':'active_siret'})
sirets2

In [ ]:
catjur2 = pd.merge(catjur,sirets2,how='inner',on=['resourceId'])
catjur2

In [ ]:
les__structures2=les__structures[['siret','etablissementSiege','dateCreationEtablissement','etatAdministratifUniteLegale','dateCreationUniteLegale','sigleUniteLegale','denominationUniteLegale','categorieJuridiqueUniteLegale','codeCommuneEtablissement']]
les__structures2=les__structures2.drop_duplicates()
catjur3 = pd.merge(catjur2,les__structures2,how='inner',on=['siret'])
catjur3

In [ ]:
filename_in = dir_output+"catjur3_"+datetime.now().strftime('%Y-%m-%d')+".xlsx"
writer = pd.ExcelWriter(filename_in, engine='xlsxwriter')
catjur3.to_excel(writer,sheet_name="data", index=False)
writer.close()

In [ ]:
headers = {
    'X-INSEE-Api-Key-Integration': inseekey,
}
def extract_siren2(numero_siren):
    global dernier_traitement_siren
    structure = {}
    sep=";"
    params = {
        'q': 'siren:'+numero_siren,
        'masquerValeursNulles' : 'true'
    }
    if os.path.isfile(dir_api+"/"+numero_siren+".json"):
        f = open(dir_api+"/"+numero_siren+".json")
        result = json.load(f)
        f.close()
    else:
        response = requests.get('https://api.insee.fr/api-sirene/3.11/siren', params=params, headers=headers)
        writeFile =open(dir_api+"/"+numero_siren+".json", 'w')
        json.dump(response.json(),writeFile)
        writeFile.close()
        result = response.json()
        time.sleep(2.02)
    if result["header"]["statut"]==200:
        if "unitesLegales" in result:
            if result["unitesLegales"][0]["nombrePeriodesUniteLegale"]>0:
                structure = pd.DataFrame(result["unitesLegales"][0]["periodesUniteLegale"])
                structure["changements"]=""
                structure["les_changements"]=""
            if "dateCreationUniteLegale" in result["unitesLegales"][0]:
                structure["dateCreationUniteLegale"]=result["unitesLegales"][0]["dateCreationUniteLegale"]
            if "identifiantAssociationUniteLegale" in result["unitesLegales"][0]:
                structure["identifiantAssociationUniteLegale"]=result["unitesLegales"][0]["identifiantAssociationUniteLegale"]
            if "sigleUniteLegale" in result["unitesLegales"][0]:
                structure["sigleUniteLegale"]=result["unitesLegales"][0]["sigleUniteLegale"]
            if "dateDernierTraitementUniteLegale" in result["unitesLegales"][0]:
                structure["dateDernierTraitementUniteLegale"]=result["unitesLegales"][0]["dateDernierTraitementUniteLegale"]
            if "categorieEntreprise" in result["unitesLegales"][0]:
                structure["categorieEntreprise"]=result["unitesLegales"][0]["categorieEntreprise"]
            if 'changementNicSiegeUniteLegale' in structure.columns:
                structure.loc[structure["changementNicSiegeUniteLegale"], 'changements'] +="Nic Siege;"
                if 'nicSiegeUniteLegale' in structure.columns:
                    structure.loc[structure["changementNicSiegeUniteLegale"], 'les_changements'] +=numero_siren+structure["nicSiegeUniteLegale"]+";"
                    structure["dern_numero_siret"]=numero_siren+structure[structure["changementNicSiegeUniteLegale"]]["nicSiegeUniteLegale"].head(1)
            if 'changementEtatAdministratifUniteLegale' in structure.columns:
                structure.loc[structure["changementEtatAdministratifUniteLegale"], 'changements'] +="État administratif;"
                if 'etatAdministratifUniteLegale' in structure.columns:
                    structure.loc[structure["changementEtatAdministratifUniteLegale"], 'les_changements'] +=structure["etatAdministratifUniteLegale"]+";"
            if 'changementCategorieJuridiqueUniteLegale' in structure.columns:
                structure.loc[structure["changementCategorieJuridiqueUniteLegale"], 'changements'] +="Catégorie juridique;"
                if 'categorieJuridiqueUniteLegale' in structure.columns:
                    structure.loc[structure["changementCategorieJuridiqueUniteLegale"], 'les_changements'] +=structure["categorieJuridiqueUniteLegale"]+";"
            if 'changementActivitePrincipaleUniteLegale' in structure.columns:
                structure.loc[structure["changementActivitePrincipaleUniteLegale"], 'changements'] +="Activité principale;"
                if 'activitePrincipaleUniteLegale' in structure.columns:
                    structure.loc[structure["changementActivitePrincipaleUniteLegale"], 'les_changements'] +=structure["activitePrincipaleUniteLegale"]+";"
            if 'changementDenominationUniteLegale' in structure.columns:
                structure.loc[structure["changementDenominationUniteLegale"], 'changements'] +="Dénomination;"
                if 'denominationUniteLegale' in structure.columns:
                    structure.loc[structure["changementDenominationUniteLegale"], 'les_changements'] +=structure["denominationUniteLegale"]+";"
            if 'changementDenominationUsuelleUniteLegale' in structure.columns:
                structure.loc[structure["changementDenominationUsuelleUniteLegale"], 'changements'] +="Dénomination usuelle;"
            if 'changementNomUniteLegale' in structure.columns:
                structure.loc[structure["changementNomUniteLegale"], 'changements'] +="Nom;"
            if 'changementNomUsageUniteLegale' in structure.columns:
                structure.loc[structure["changementNomUsageUniteLegale"], 'changements'] +="Nom d'usage;"
            if 'changementSocieteMissionUniteLegale' in structure.columns:
                structure.loc[structure["changementSocieteMissionUniteLegale"], 'changements'] +="Société à mission;"
            if 'changementEconomieSocialeSolidaireUniteLegale' in structure.columns:
                structure.loc[structure["changementEconomieSocialeSolidaireUniteLegale"], 'changements'] +="Champ de l'économie sociale et solidaire;"
            if 'changementCaractereEmployeurUniteLegale' in structure.columns:
                structure.loc[structure["changementCaractereEmployeurUniteLegale"], 'changements'] +="Caractère employeur;"
            structure["changements"] = structure["changements"].str[:-1]
            structure["les_changements"] = structure["les_changements"].str[:-1]
            structure['index_column'] = structure.index+1
            structure['index_column'] = result["unitesLegales"][0]["nombrePeriodesUniteLegale"]+1-structure['index_column']
        structure["numero_siren"] = numero_siren
        structure["statut"] = result["header"]["statut"]
        if numero_siren in siren_paysage_unique['siren'].values:
            id_paysage = siren_paysage_unique.loc[(siren_paysage['siren']==numero_siren), 'resourceId'].max()
            structure["paysage"] = "https://paysage.enseignementsup-recherche.gouv.fr/structures/"+id_paysage+"/presentation"
            structure["id_paysage"] = id_paysage
        else:
            structure["paysage"] = "https://paysage.enseignementsup-recherche.gouv.fr/rechercher?query="+numero_siren
        structure["annuaire"] = "https://annuaire-entreprises.data.gouv.fr/entreprise/"+numero_siren
        structure["ods"] = "https://data.enseignementsup-recherche.gouv.fr/explore/dataset/fr-esr-maj_structures_sirene/table/?sort=datedebut&refine.numero_siren="+numero_siren
        structure["dernier_traitement"]=dernier_traitement_siren
        return structure

In [ ]:
dernier_traitement_siren=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
maj_structure4 = extract_siren2("438178725")
maj_structure4

In [ ]:
maj_structures=pd.DataFrame(columns = ["numero_siren","statut"])
i=0
dernier_traitement_siren=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
for ind in les_sirens.index:
    numero_siren = les_sirens[ind]
    i=i+1
    if i%500==0:
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S')+" "+str(i)+"/{:0.1f} %".format(i*100/len(les_sirens.index)))
    maj_structure = extract_siren2(numero_siren)
    maj_structure = pd.DataFrame(maj_structure)
    if (maj_structures.empty) & (not maj_structure.empty):
        maj_structures=maj_structure
    if not maj_structure.empty:
        maj_structures = pd.concat([maj_structures, maj_structure], axis=0, join='outer')
print("Terminé")

In [ ]:
selection = maj_structures[(maj_structures["dateDebut"]=="2207-12-21")].index
maj_structures[(maj_structures["dateDebut"]=="2207-12-21")]

In [ ]:
#selection = maj_structures[(maj_structures["dateDebut"]=="2207-12-21")].index
#maj_structures.loc[selection, 'dateDebut']="2012-07-26"
maj_structures=pd.merge(maj_structures,les__adresse,how='left',on=['dern_numero_siret'])
maj_structures=pd.merge(maj_structures,dateDernierTraitementEtablissement,how='left',on=['numero_siren'])
maj_structures=pd.merge(maj_structures,categoriesJuridiques,how='left',on=['categorieJuridiqueUniteLegale'])
maj_structures=pd.merge(maj_structures,activitePrincipaleUniteLegale,how='left',on=['nomenclatureActivitePrincipaleUniteLegale','activitePrincipaleUniteLegale'])
maj_structures=pd.merge(maj_structures,siret_paysage,how='left',on=['dern_numero_siret'])
maj_structures=pd.merge(maj_structures,maj_structures_sirene_prec,how='left',on=['numero_siren','index_column'])
selection = maj_structures[maj_structures["Nouveau"]!="Non"].index
maj_structures.loc[selection, 'Nouveau']="Oui"
maj_structures.loc[selection, 'Ajout']=datetime.now().strftime('%Y-%m-%d')
maj_structures.loc[(maj_structures["Siret_dans_paysage"].isnull()&~maj_structures["dern_numero_siret"].isnull()), 'Siret_dans_paysage']= "Non"
maj_structures

In [ ]:
maj_structures.columns

In [ ]:
maj_structures_prec=maj_structures[["numero_siren","index_column","Ajout"]].drop_duplicates()
maj_structures_prec["Nouveau"]="Non"
maj_structures_prec.to_parquet(dir_data+"maj_structures_sirene_prec.parquet")

In [ ]:
filename_in = dir_output+"maj_structures_sirene_"+datetime.now().strftime('%Y-%m-%d')+".xlsx"
writer = pd.ExcelWriter(filename_in, engine='xlsxwriter', engine_kwargs={'options':{'strings_to_urls': False}})
maj_structures.to_excel(writer,sheet_name="data", index=False)
writer.close()
ftp_server = ftplib.FTP(ftp_HOSTNAME, ftp_USERNAME, ftp_PASSWORD)
filename_out = "maj_structures_sirene.xlsx"
with open(filename_in, "rb") as file:
    ftp_server.storbinary(f"STOR {filename_out}", file)
print(ftp_server.dir())
ftp_server.quit()
maj_structures

In [ ]:
response = requests.get(
    'https://mesr.opendatasoft.com/api/automation/v1.0/datasets/?limit=999', 
    headers={"Authorization": "apikey "+apikeyods})
result=response.json()
df = pd.DataFrame(result["results"])
od = df[df['dataset_id'].str.contains('fr-esr-maj_structures_sirene')]
od

In [ ]:
dataset_uid="da_lv93cw"

In [ ]:
try:
    responseF = requests.post('https://mesr.opendatasoft.com/api/automation/v1.0/datasets/'+dataset_uid+'/publish/', headers={"Authorization": "apikey "+apikeyods})
    responseF.raise_for_status()
except requests.exceptions.HTTPError as errh:
    print(dataset_uid+" > KO : publish ", responseF.status_code)
print("fin")

In [ ]:
Rna_sirene=maj_structures[(~maj_structures["identifiantAssociationUniteLegale"].isnull())][["numero_siren","identifiantAssociationUniteLegale"]].drop_duplicates()
Rna_sirene

In [ ]:
RNAcrois = pd.merge(Rna_sirene,Rna_paysage,how='left',on=['identifiantAssociationUniteLegale'])
RNAcrois[(RNAcrois["Rna_dans_paysage"].isnull())]

In [ ]:
maj_structures.columns

In [ ]:
structures_legcat

In [ ]:
structures.columns

In [ ]:
maj_structures_prec

In [ ]:
heure_fin=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print("début : "+heure_debut)
print("fin : "+heure_fin)